In [575]:
#Shane Stevens
from os import walk
import numpy as np
import string



In [577]:
#This cell is for the "settings" of the program

#The program causes overflow warnings when working with the testing data, so I set this to ignore it so the output looks nicer
np.seterr(over = 'ignore')

#directory of the data being used
#current opptions are 'input_data\\Emails', 'input_data\\Presidential_Speeches', and 'input_data\\Combined'
data_dir = 'input_data\\Combined'

#overall percentage of data used (mainly for saving time when working with large datasets)
#must be between 0 and 1.0
percent_used = 1.0

#percent of used data for training and testing respectively
#percent_traning must be between 0 and 1, and the two values must add up to 1
percent_training = .75
percent_testing = 1.0-percent_training

#used to filter out words that may skew data, such as "the" or "and"
filtered_words = []


In [579]:
#Gathers data from the text files
classes = []
allfiles = []
isStart = True
#gathers all of the files in the given directory, as well as their paths
for(dirpath,dirnames,filenames) in walk(data_dir):
    if isStart == True:
        classes = dirnames
        isStart = False;
    if(filenames != []):
        allfiles.append((dirpath,filenames))

#sort classes by length to prevent errors where class names contain another class name
classes.sort(key = len, reverse = True)

print(classes)
print(len(allfiles))

text_data_training = []
text_data_testing = []
amount_per_class = [0] * len(classes)
classifier = ''
for file_list in allfiles:
    for classif in classes:
        if classif in file_list[0]:
            classifier = classif
    np.random.shuffle(file_list[1])
    num_files_used = round(len(file_list[1])*percent_used)
    for i in range(0, num_files_used):
        with open(file_list[0] + '\\' + file_list[1][i], encoding = 'latin-1') as file:
            text = file.read()
            text.lower()
            text.translate(str.maketrans('','',string.punctuation))
            if i < round(num_files_used * percent_training):
                text_data_training.append((text,classifier))
                amount_per_class[classes.index(classifier)]+= 1
            else:
                text_data_testing.append((text,classifier))
                
print("number of training data: " + str(len(text_data_training)))
print(amount_per_class)
print("number of testing data: " + str(len(text_data_testing)))

['democratic', 'republican', 'Spam', 'Ham']
5
number of training data: 467
[9, 9, 225, 224]
number of testing data: 157


In [581]:
#creates model based off of training data
word_dict = {}

for text in text_data_training:
    for word in text[0].split():
        if word not in word_dict:
            word_dict[word] = {}
            for classif in classes:
                word_dict[word][classif] = 0
        word_dict[word][text[1]] += 1
print(word_dict)
                

{'day': {'democratic': 31, 'republican': 15, 'Spam': 14, 'Ham': 23}, 'office': {'democratic': 8, 'republican': 14, 'Spam': 35, 'Ham': 9}, 'president': {'democratic': 20, 'republican': 26, 'Spam': 16, 'Ham': 3}, 'state': {'democratic': 48, 'republican': 33, 'Spam': 21, 'Ham': 4}, 'ability': {'democratic': 5, 'republican': 9, 'Spam': 8, 'Ham': 1}, 'constitution': {'democratic': 25, 'republican': 11, 'Spam': 0, 'Ham': 0}, 'god': {'democratic': 10, 'republican': 20, 'Spam': 26, 'Ham': 1}, 'father': {'democratic': 2, 'republican': 3, 'Spam': 14, 'Ham': 0}, 'wisdom': {'democratic': 3, 'republican': 5, 'Spam': 0, 'Ham': 0}, 'duty': {'democratic': 13, 'republican': 21, 'Spam': 5, 'Ham': 1}, 'manner': {'democratic': 4, 'republican': 0, 'Spam': 7, 'Ham': 3}, 'friendship': {'democratic': 2, 'republican': 1, 'Spam': 0, 'Ham': 0}, 'people': {'democratic': 99, 'republican': 85, 'Spam': 10, 'Ham': 11}, 'generation': {'democratic': 13, 'republican': 9, 'Spam': 0, 'Ham': 1}, 'election': {'democratic': 

In [583]:
#test model using testing data
predicted_class = [0] * len(classes)
actual_class = [0] * len(classes)
predicted_vs_actual_class = []
probabilities = []
for i in range(0, len(classes)):
    probabilities.append(amount_per_class[i]/len(text_data_training))
    
index = 0
word_probabilities = [0] * len(classes)
for text in text_data_testing:
    for i in range(0, len(classes)):
        word_probabilities[i] = (probabilities[i])
    text_dict = {}
    for word in text[0].split():
        if word not in filtered_words:
            if word not in text_dict.keys():
                text_dict[word] = 1
            else:
                text_dict[word] += 1
    for word in text_dict.keys():
        if word in word_dict.keys():
            for i in range(len(classes)):
                word_probabilities[i] = word_probabilities[i] * ((text_dict[word] * (word_dict[word][classes[i]] + 1))/(len(text_data_training) + len(word_dict)))
        else:
            for i in range(len(classes)):
                word_probabilities[i] = word_probabilities[i] * ((text_dict[word] *1)/(len(text_data_training) + len(word_dict)))
    highest_score = word_probabilities[0]
    predicted_class_index = 0;
    for i in range(0,len(classes)):
        if word_probabilities[i] > highest_score:
            predicted_class_index = i
        if text[1] == classes[i]:
            actual_class[i] += 1
    predicted_class[predicted_class_index] += 1
    predicted_vs_actual_class.append((classes[predicted_class_index],text[1]))

print(predicted_class)
print(actual_class)
print(predicted_vs_actual_class)


[64, 0, 14, 79]
[3, 3, 75, 76]
[('democratic', 'democratic'), ('democratic', 'democratic'), ('democratic', 'democratic'), ('Ham', 'Ham'), ('democratic', 'Ham'), ('Ham', 'Ham'), ('Ham', 'Ham'), ('democratic', 'Ham'), ('Ham', 'Ham'), ('Ham', 'Ham'), ('Ham', 'Ham'), ('Ham', 'Ham'), ('Ham', 'Ham'), ('Ham', 'Ham'), ('Ham', 'Ham'), ('Ham', 'Ham'), ('Ham', 'Ham'), ('Ham', 'Ham'), ('Ham', 'Ham'), ('Ham', 'Ham'), ('Ham', 'Ham'), ('Ham', 'Ham'), ('democratic', 'Ham'), ('Ham', 'Ham'), ('Ham', 'Ham'), ('Ham', 'Ham'), ('democratic', 'Ham'), ('Ham', 'Ham'), ('Ham', 'Ham'), ('Ham', 'Ham'), ('Ham', 'Ham'), ('democratic', 'Ham'), ('Ham', 'Ham'), ('Ham', 'Ham'), ('Ham', 'Ham'), ('Ham', 'Ham'), ('Ham', 'Ham'), ('Ham', 'Ham'), ('Ham', 'Ham'), ('Ham', 'Ham'), ('democratic', 'Ham'), ('Ham', 'Ham'), ('democratic', 'Ham'), ('Ham', 'Ham'), ('Ham', 'Ham'), ('Spam', 'Ham'), ('Ham', 'Ham'), ('Ham', 'Ham'), ('Ham', 'Ham'), ('Ham', 'Ham'), ('Ham', 'Ham'), ('Ham', 'Ham'), ('Ham', 'Ham'), ('Ham', 'Ham'), ('Ham', 'Ham

In [585]:
#print scores for the data

#calculate confusion matrix values
confusion_matrix_values = [(0,0)]*len(classes)
for pair in predicted_vs_actual_class:
    if pair[0] == pair[1]:
        new_tuple = (confusion_matrix_values[classes.index(pair[0])][0] + 1,confusion_matrix_values[classes.index(pair[0])][1])
        confusion_matrix_values[classes.index(pair[0])] = new_tuple
    else:
        new_tuple = (confusion_matrix_values[classes.index(pair[0])][0],confusion_matrix_values[classes.index(pair[0])][1]+1)
        confusion_matrix_values[classes.index(pair[0])] = new_tuple
for classif in classes:
    print("True " + classif + " values: " + str(confusion_matrix_values[classes.index(classif)][0]))
    print("False " + classif + " values: " + str(confusion_matrix_values[classes.index(classif)][1]))

true_sum = 0;
for tup in confusion_matrix_values:
    true_sum += tup[0]
    
accuracy = true_sum/len(text_data_testing)

print("\n accuracy: " + str(accuracy))


True democratic values: 3
False democratic values: 61
True republican values: 0
False republican values: 0
True Spam values: 13
False Spam values: 1
True Ham values: 65
False Ham values: 14

 accuracy: 0.5159235668789809
